In [1]:
from AREgym import AREEnv

import gym


from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.env_checker import check_env
# env = env = gym.make('Pendulum-v0')
# env = AREEnv(250,10)
import numpy as np


In [2]:
env = AREEnv(250, 10, num_laser_scan=10, heuristic_dist=10, save_map=True)
check_env(env)


In [3]:
model = PPO("MlpPolicy",
            env,
            verbose=0,
            n_steps=1024,
            tensorboard_log='sb_runs/1024',
            seed=1
            )
# model = PPO("MlpPolicy", env, verbose=1, gamma=0.9)
model.learn(total_timesteps=500000)
model.save("stable_baselines3_ppo")

In [3]:
model = PPO.load("stable_baselines3_ppo")

In [4]:
model.policy

ActorCriticPolicy(
  (features_extractor): FlattenExtractor(
    (flatten): Flatten(start_dim=1, end_dim=-1)
  )
  (mlp_extractor): MlpExtractor(
    (shared_net): Sequential()
    (policy_net): Sequential(
      (0): Linear(in_features=20, out_features=64, bias=True)
      (1): Tanh()
      (2): Linear(in_features=64, out_features=64, bias=True)
      (3): Tanh()
    )
    (value_net): Sequential(
      (0): Linear(in_features=20, out_features=64, bias=True)
      (1): Tanh()
      (2): Linear(in_features=64, out_features=64, bias=True)
      (3): Tanh()
    )
  )
  (action_net): Linear(in_features=64, out_features=1, bias=True)
  (value_net): Linear(in_features=64, out_features=1, bias=True)
)

In [8]:
obs = env.reset()

for i in range(1024):
    action, _states = model.predict(obs)
    obs, rewards, dones, info = env.step(action)
    print(action)
    # env.render()
env.render()

[1.]
[1.]
[0.35920262]
[1.]
[1.]
[0.85495305]
[1.]
[1.]
[1.]
[1.]
[0.8140905]
[1.]
[0.7779057]
[0.74474096]
[0.85774803]
[0.8994705]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[0.94191855]
[1.]
[1.]
[0.80099654]
[1.]
[1.]
[0.89023143]
[0.77970815]
[1.]
[1.]
[1.]
[0.82649267]
[1.]
[0.7769591]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[0.8953997]
[1.]
[1.]
[1.]
[0.6627934]
[1.]
[0.78843975]
[0.7443933]
[1.]
[1.]
[1.]
[0.8055015]
[0.84278166]
[0.8663578]
[1.]
[1.]
[0.7376207]
[1.]
[1.]
[1.]
[1.]
[0.7628426]
[0.77829176]
[0.9495568]
[0.8425132]
[1.]
[0.626446]
[0.30467796]
[0.9730686]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[0.9327191]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[0.74620396]
[0.80802006]
[1.]
[1.]
[1.]
[0.81206197]
[0.8418285]
[1.]
[1.]
[1.]
[0.8061439]
[1.]
[1.]
[1.]
[0.9920151]
[1.]
[0.906981]
[0.5666757]
[-0.46529865]
[0.90665734]
[0.4807232]
[-0.76968056]
[-0.773697]
[-0.5065514]
[0.91872567]
[0.33228087]
[0.01993176]
[-0.63287115]
[0.2691566]
[0.8140387]
[0.82728124]
[0.5

In [ ]:
# import gym

# from stable_baselines3 import PPO
# from stable_baselines3.common.env_util import make_vec_env

# # Parallel environments
# env = make_vec_env("CartPole-v1", n_envs=4)

# model = PPO("MlpPolicy", env, verbose=1)
# model.learn(total_timesteps=25000)
# model.save("ppo_cartpole")

# del model # remove to demonstrate saving and loading

# model = PPO.load("ppo_cartpole")

# obs = env.reset()
# while True:
#     action, _states = model.predict(obs)
#     obs, rewards, dones, info = env.step(action)
#     env.render()